In [1]:
import os
import numpy as np
import pandas as pd
import csv

In [2]:
# Rutas
ROOT_PATH = 'C:/Users/pedro/ds-geo'
CONFIG_PATH = ROOT_PATH + '/config'
DATA_PATH = ROOT_PATH + '/data'
MYTRACK_PATH = ROOT_PATH + '/data/mytrack'

DATASET_MYTRACK_P = 'dataset_mytrack_points.pkl'
DATASET_MYTRACK_M = 'dataset_mytrack_markers.pkl'

os.chdir(ROOT_PATH)

In [3]:
# Carga del fichero de la aplicación MyTrack

def LoadMytrackCSV(s_file):
    
    l_lines = []
    s_pathfile = MYTRACK_PATH + '/' + s_file
    
    with open(s_pathfile,'r') as f:
        x = csv.reader(f)
        new = list(x)
        for i, line in enumerate(new):
            if (len(line) == 0):
                l_lines.append(i + 1)
        l_lines.append(len(new))
        f.close()
        
        if len(l_lines) >= 3:
            # El formato del fichero es correcto. Lo podemos cargar.
            df_points = pd.read_csv(s_pathfile, sep=',', skiprows=l_lines[1], encoding='UTF-8', engine='python')
            df_markers = pd.read_csv(s_pathfile, sep=',', skiprows=l_lines[0], skipfooter=l_lines[2] - l_lines[1], encoding='UTF-8', engine='python')
            df_markers.rename({'Name': 'Etiqueta'}, axis=1, inplace=True)
    
    return (df_points, df_markers)


In [4]:
def LoadMyTrack():
    
    # Obtengo los ficheros de MyTrack a cargar
    files_list = []
    for file in os.listdir(MYTRACK_PATH):
        if file.endswith(".csv"):
            files_list.append(file)

    df_points = pd.DataFrame(columns=['Location', 'Time', 'Latitude', 'Longitude', 'Altitude', 'Accuracy(m)', 'Speed(m/s)', 'Bearing', 'CodigoCultivo'])
    df_markers = pd.DataFrame(columns=['Marker', 'Time', 'Latitude', 'Longitude', 'Etiqueta', 'CodigoCultivo'])
    
    # Leo la información de los ficheros de MyTrack y la añado al dataframe
    for file in files_list:
        df_new_p, df_new_m = LoadMytrackCSV(file)
        
        s_tipo = file.split('-', 2)[0].upper()
        s_propietario = file.split('-', 2)[1].upper()
        df_new_p['CodigoCultivo'] = s_tipo
        df_new_p['CodigoPropietario'] = s_propietario
        df_new_p['Origen'] = file
        df_new_m['CodigoCultivo'] = s_tipo
        df_new_m['Origen'] = file
        df_new_m['CodigoPropietario'] = s_propietario
        
        df_points = pd.concat([df_points, df_new_p], axis=0, ignore_index=True)
        df_markers = pd.concat([df_markers, df_new_m], axis=0, ignore_index=True)
    
    # Leo el fichero de configuración y completo los datos de MyTrack
    df_infoC = pd.read_csv(CONFIG_PATH + '/cultivos.cfg', sep=',', encoding='UTF-8', na_filter= False)
    df_infoP = pd.read_csv(CONFIG_PATH + '/propietarios.cfg', sep=',', encoding='UTF-8', na_filter= False)
    
    df_points = pd.merge(df_points, df_infoC, on=['CodigoCultivo'], how='left')
    df_points = pd.merge(df_points, df_infoP, on=['CodigoPropietario'], how='left')
    df_points.fillna('', inplace=True)
    df_points.drop(['CodigoCultivo', 'CodigoPropietario'], axis=1, inplace=True)

    df_markers = pd.merge(df_markers, df_infoC, on=['CodigoCultivo'], how='left')
    df_markers = pd.merge(df_markers, df_infoP, on=['CodigoPropietario'], how='left')
    df_markers.fillna('', inplace=True)
    df_markers.drop(['CodigoCultivo', 'CodigoPropietario'], axis=1, inplace=True)
    
    return (df_points, df_markers)


In [5]:
# Función que guarda un dataset

def SaveDataset(df, name='dataset.pkl'):
    df.to_pickle(DATA_PATH + '/' + name)
 

In [6]:
# Función que fusiona la información del Catastro con la propia de la empresa 
# para generar el dataset que se visualizará postariormente

def GenerateDatasetMyTrack():
    # Cargamos los puntos de interés
    df_mytrack_points, df_mytrack_markers = LoadMyTrack()

    # Volcamos a disco los datasets
    SaveDataset(df_mytrack_points, DATASET_MYTRACK_P)
    SaveDataset(df_mytrack_markers, DATASET_MYTRACK_M)


In [7]:
# Función que carga el dataset de los markers

def LoadDatasetMyTrack():
    df_points = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_P)
    df_markers = pd.read_pickle(DATA_PATH + '/' + DATASET_MYTRACK_M)
    return (df_points, df_markers)


## Ejecución

In [8]:
# Generación del dataset de puntos
GenerateDatasetMyTrack()

In [9]:
p, m = LoadDatasetMyTrack()

In [10]:
p

,Location,Time,Latitude,Longitude,Altitude,Accuracy(m),Speed(m/s),Bearing,Origen,NombreCultivo,ColorCultivo,NombrePropietario,ColorPropietario
0,244,2022-10-09T06:48:58.000Z,39.683117,-2.919280,815.3,8.517,0.351508,88.269450,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
1,245,2022-10-09T07:00:31.286Z,39.683083,-2.919390,815.9,6.336,0.198902,272.707920,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
2,246,2022-10-09T07:00:34.548Z,39.683080,-2.919450,816.0,6.237,0.215731,277.219540,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
3,247,2022-10-09T07:00:43.524Z,39.683075,-2.919511,816.2,6.517,0.021595,267.214020,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
4,248,2022-10-09T07:00:51.941Z,39.683113,-2.919577,815.7,5.981,0.073619,304.258540,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301,1545,2022-10-09T10:31:09.527Z,39.682896,-2.919429,811.8,6.729,0.759096,353.357030,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
1302,1546,2022-10-09T10:31:15.543Z,39.682964,-2.919416,813.1,6.974,0.734434,5.837004,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
1303,1547,2022-10-09T10:31:22.041Z,39.683030,-2.919369,814.8,6.525,0.703462,15.288898,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
1304,1548,2022-10-09T10:31:27.918Z,39.683070,-2.919330,815.0,6.194,0.325375,16.487670,cza--MyTracks_Caza_2022-10-09.csv,Caza,darkgreen,,
